In [1]:
# dependencies

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import datetime
import plotly.express as px

from sqlalchemy import create_engine

pd.set_option('display.max_columns', None)

In [38]:
# database connection

db_password = "UnitCircle42!"
db_user = "postgres"
db_name = "dot"
endpoint = "awakedb.cre3f7yk1unp.us-west-1.rds.amazonaws.com"

connection_string = f"postgresql://{db_user}:{db_password}@{endpoint}:5432/{db_name}"
engine = create_engine(connection_string)

In [49]:
dot = pd.read_sql('SELECT * FROM invoice_clean;', con = engine)
# unl = pd.read_sql('SELECT * FROM unleashed_clean;', con = engine)

# unl.columns = ['month','year','date','customer','item','qty','cad','usd','sale_origin','market_segment','parent_customer']
# dot.columns = ['month','year','customer','date','item','qty','usd','cad','sale_origin','parent_customer','market_segment']

In [55]:
dot.parent_customer.unique()

array(['A & R WHOLESALE DNU', 'A.C.C. WHOLESALE', 'A.E. WEASE INC.',
       'ABL Wholesale', 'ACE ENDICO CORP.', 'Albanese',
       'All Fresh Products', 'Amcon', 'AMCON DIST/CROSSVILLE',
       'Core-Mark', 'AMSTER KIRTZ DIV OF ABL', 'AVI FOODSYSTEMS INC',
       'AWAKE CHOC/ S.ABRAHAMS', 'AWG/KANSAS CITY', 'AWG/OKLAHOMA CITY',
       'BAR MART/BENEFIELD ENT.', "BELL'S WHOLESALE GROCERY",
       'BEN E. KEITH/DFW', 'BEN E. KEITH/LITTLE R-DNU',
       'BEN E. KEITH/SOUTHEAST', 'Bloom Wholesale', 'Blue Market Group',
       'BTO GROUP-INDIANA', 'BUY BULK AMERICA', 'C&S/HANSEN-HAWAII-OAHU',
       'CD Hartnett', 'CM Tanner Grocery', 'Capitol Distributing',
       'Cash-WA of Fargo', 'CASH-WA DISTR. OF KEARNEY',
       'Century Distributors', 'Certco', 'CERTCO FOODS/FEMRITE',
       'Chambers & Owen', 'Coast Brothers', 'COFFEE & FILTERS DIRECT',
       'J Polep', 'CORE-MARK/HAYWARD', 'CORE-MARK/LAS VEGAS',
       'COSGROVE DISTRIBUTORSINC', 'CP FOODS', 'CVS PHARMACY/ BESSEMER',
       'CV

In [26]:
new_order = ['month', 'year', 'date', 'customer', 'item',
       'qty', 'cad', 'usd', 'sale_origin', 'market_segment',
       'parent_customer']

for column in new_order:
    dot[column] = dot.pop(column)

In [29]:
## concat datasets

lvl2 = pd.concat([dot,unl]).sort_values(by='date',ascending=False).reset_index(drop=True)
lvl2 = lvl2[lvl2.customer != 'DOT Foods, Inc.']
lvl2 = lvl2.convert_dtypes()
lvl2.sample()

,month,year,date,customer,item,qty,cad,usd,sale_origin,market_segment,parent_customer
168901,August,2020,2020-08-12,Amazon FBM,50ct Change Maker - AWAKE Chocolate Bites USA ...,1.0,22.13,16.6,unl,direct,customer_name


In [32]:
 lvl2.groupby(['year','month','parent_customer'],as_index=False)[['qty','usd']].sum()

,year,month,parent_customer,qty,usd
0,2016,April,AMSTER KIRTZ DIV OF ABL,9.0,706.32
1,2016,April,AVI FOODSYSTEMS INC,60.0,2592.0
2,2016,April,CORE-MARK/HAYWARD,2.0,156.96
3,2016,April,CORE-MARK/LAS VEGAS,3.0,276.96
4,2016,April,Core-Mark,155.0,13036.32
...,...,...,...,...,...
5525,2024,January,US Foods,28.0,3171.56
5526,2024,January,USF/CHICAGO,10.0,274.54
5527,2024,January,Vistar,558.0,73177.14
5528,2024,January,Vistar Retail,1471.0,59087.14


In [23]:
grouped_lvl2 = (
    lvl2.groupby(['customer','parent_customer','year','month','sale_origin','market_segment','item'],as_index=False,observed=False)
    [['qty','usd','cad']]
    .sum(numeric_only = True)
    .sort_values(by='usd',ascending=False)
    .round(0)
).reset_index(drop=True)

grouped_lvl2[grouped_lvl2.customer != 'DOT Foods, Inc.']

,customer,parent_customer,year,month,sale_origin,market_segment,item,qty,usd,cad
40,Vistar/ VSA Corporate,customer_name,1970-01-01 00:00:00.000002018,February,unl,direct,6 x 50ct Change Maker - AWAKE Chocolate Bites ...,1000.0,114997.0,153329.0
42,Vistar/ VSA Corporate,customer_name,1970-01-01 00:00:00.000002018,February,unl,direct,6 x 50ct Change Maker - AWAKE Chocolate Bites ...,979.0,112582.0,150109.0
47,Vistar/ VSA Corporate,customer_name,1970-01-01 00:00:00.000002018,March,unl,direct,6 x 12 Pack - AWAKE Caffeinated Chocolate Bars...,1407.0,105817.0,141090.0
93,Vistar/ VSA Corporate,customer_name,1970-01-01 00:00:00.000002019,November,unl,direct,6 x 50ct Change Maker - AWAKE Chocolate Bites ...,614.0,73309.0,97745.0
107,VISTAR-GEORGIA,Vistar,1970-01-01 00:00:00.000002023,September,dot,VENDING,CAFFEINATED CHOCOLATE BITES SINGLES CARAMEL,465.0,65565.0,87201.0
...,...,...,...,...,...,...,...,...,...,...
44609,E-TAILER INC,E-TAILER INC,1970-01-01 00:00:00.000002016,September,dot,ONLINE DISTRIBUTORS,CAFFEINATED CHOCOLATE BITES SINGLES,-50.0,-6000.0,-7980.0
44610,VISTAR-NORTHERN CALIFORNI,Vistar,1970-01-01 00:00:00.000002023,December,dot,VENDING,AWAKE CHOCOLATE AWAKE CAFF MILK CHOC- 6X12PK M...,-80.0,-6843.0,-9101.0
44611,VISTAR-NORTHERN CALIFORNI,Vistar,1970-01-01 00:00:00.000002023,December,dot,VENDING,CAFFEINATED CHOCOLATE BITES SINGLES PB CHOC,-64.0,-9024.0,-12002.0
44612,VISTAR-NORTHERN CALIFORNI,Vistar,1970-01-01 00:00:00.000002020,March,dot,VENDING,CAFFEINATED CHOCOLATE BITES SINGLES,-120.0,-11520.0,-15322.0


In [7]:
melted_lvl2 = grouped_lvl2.melt(id_vars=['customer','parent_customer','year','month','sale_origin','market_segment','item'],value_vars=['qty','usd','cad'])
melted_lvl2.head()

,customer,parent_customer,year,month,sale_origin,market_segment,item,variable,value
0,VISTAR-GEORGIA,Vistar,2023,September,dot,VENDING,CAFFEINATED CHOCOLATE BITES SINGLES CARAMEL,qty,465.0
1,VISTAR-GEORGIA,Vistar,2021,October,dot,VENDING,CAFFEINATED CHOCOLATE BITES SINGLES CARAMEL,qty,446.0
2,VISTAR-FLORIDA,Vistar,2018,August,dot,VENDING,AWAKE CHOCOLATE AWAKE CHOC CARAMEL- 6X12PK MASTER,qty,690.0
3,VISTAR-FLORIDA,Vistar,2019,March,dot,VENDING,AWAKE BARS- PB CHOC BBITE - 6X12PK MASTER,qty,640.0
4,H.E.BUTT GROCERY,H.E.BUTT GROCERY,2019,November,dot,GROCERY,AWAKE CHOCOLATE AWAKE CHOC CARAMEL- 6X12PK MASTER,qty,634.0


In [8]:
import plotly.express as px

In [10]:
px.bar(lvl2[lvl2.],
      x='parent_customer',
      y='usd')

ValueError: Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['customer', 'parent_customer', 'year', 'month', 'sale_origin', 'market_segment', 'item', 'variable', 'value'] but received: usd